In [1]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("all-MiniLM-L6-v2")
print("Module loaded successfully!")


C:\Users\HP\anaconda3\envs\langchain_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\HP\anaconda3\envs\langchain_env\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\HP\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. I

Module loaded successfully!


In [3]:



# auditor_agent_intelligent.py
import json
from sentence_transformers import SentenceTransformer
import numpy as np
import os

# === Config ===
MODEL_NAME = "all-MiniLM-L6-v2"  # model pour embeddings
SIM_HIGH = 0.75
SIM_MEDIUM = 0.5

# === Charger le modèle d'embeddings ===
model = SentenceTransformer(MODEL_NAME)

# === Chemins ===
researcher_json_path = "C:/Users/HP/aibuilders--main/outputs/researcher_output_chroma.json"  # chemin vers output Researcher
auditor_output_path = "auditor_output.json"  # output pour Generator Agent

# === Charger input men Researcher Agent ===
with open(researcher_json_path, "r", encoding="utf-8") as f:
    researcher_output = json.load(f)

top_5_passages = researcher_output["top_5_passages"]

# === Nouvelle regulation text (input manuel ou automatique) ===
new_regulation_text = "All passwords must follow strict security guidelines"

# === Calcul des embeddings ===
new_rule_emb = model.encode([new_regulation_text], convert_to_numpy=True)

results = []

for passage in top_5_passages:
    policy_text = passage["excerpt"]
    policy_emb = model.encode([policy_text], convert_to_numpy=True)
    
    # Similarité cosine
    sim_score = np.dot(policy_emb, new_rule_emb.T) / (
        np.linalg.norm(policy_emb) * np.linalg.norm(new_rule_emb)
    )
    sim_score = sim_score[0][0]
    
    # Déterminer severity
    if sim_score >= SIM_HIGH:
        severity = "HIGH"
        summary = "Conflict detected with high similarity"
    elif sim_score >= SIM_MEDIUM:
        severity = "MEDIUM"
        summary = "Potential conflict detected"
    else:
        severity = "LOW"
        summary = "No major conflict detected"
    
    results.append({
        "policy_id": passage["file"],
        "severity": severity,
        "divergence_summary": summary,
        "conflicting_policy_excerpt": policy_text,
        "new_rule_excerpt": new_regulation_text,
        "recommendation": "Review and update internal policy if needed"
    })

# === Sauvegarder output JSON pour Generator Agent ===
os.makedirs("outputs", exist_ok=True)
with open(os.path.join("outputs", auditor_output_path), "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=4)

print(f"Auditor analysis saved to outputs/{auditor_output_path}")



Auditor analysis saved to outputs/auditor_output.json
